In [1]:
import pandas as pd
import glob
import re
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import gc
from scipy.stats import nct, laplace_asymmetric ,genhyperbolic
pd.set_option('display.max_rows', None)
from sys import platform

In [2]:
df = pd.read_csv('data/tickers.csv',index_col=0)    
df.head()

,industry,sub_industry,type,floor,isin,status,companyName,companyNameEng,shortName,listedDate,delistedDate,companyId,taxCode,shortNameEng,indexCode
PVD,Oil & Gas,"Oil Equipment, Services & Distribution",STOCK,HOSE,VN000000PVD2,listed,Tổng Công ty Cổ phần Khoan và Dịch vụ khoan dầ...,PetroVietNam Drilling and Service Corporation,DV khoan D.khí,12/5/2006,NaN,429,302495126,PetroVietNam,NaN
PVC,Oil & Gas,"Oil Equipment, Services & Distribution",STOCK,HNX,VN000000PVC4,listed,Tổng công ty Hóa chất và Dịch vụ Dầu khí - CTC...,Petrovietnam Chemical and Services Corporation,Hóa chất & DV dầu khí,11/15/2007,NaN,1101,100150873,PVChem,HNX30
PLX,Oil & Gas,Oil & Gas Producers,STOCK,HOSE,VN000000PLX1,listed,Tập đoàn Xăng dầu Việt Nam,Vietnam National Petroleum Group,Xăng dầu Việt Nam,4/21/2017,NaN,199,100107370,Petrolimex,VN30
PVS,Oil & Gas,"Oil Equipment, Services & Distribution",STOCK,HNX,VN000000PVS0,listed,Tổng Công ty Cổ phần Dịch vụ Kỹ thuật Dầu khí ...,Petrovietnam Technical Services Corporation,DV KT D.khí VN,9/20/2007,NaN,396,100150577,Petro Technical Services,HNX30
PVB,Oil & Gas,"Oil Equipment, Services & Distribution",STOCK,HNX,VN000000PVB6,listed,Công ty Cổ phần Bọc Ống Dầu khí Việt Nam,PetroVietnam Coating Joint Stock Company,Bọc Ống D.khí VN,12/24/2013,NaN,3827,600393680,PetroVN Coating,NaN


In [6]:
df.groupby(by=['industry']).count().sort_values(by=['type'],ascending=False)

,sub_industry,type,floor,isin,status,companyName,companyNameEng,shortName,listedDate,delistedDate,companyId,taxCode,shortNameEng,indexCode
industry,,,,,,,,,,,,,,
Construction & Materials,165,165,165,152,165,165,165,150,165,19,165,162,151,10
Industrial Goods & Services,113,113,113,109,113,113,112,109,113,6,113,113,111,2
Real Estate,0,77,77,75,77,77,77,75,77,3,77,77,74,7
Food & Beverage,66,66,66,56,66,66,66,56,66,15,66,65,54,5
Basic Resources,53,53,53,51,53,53,53,51,53,6,53,53,47,4
Utilities,0,50,50,48,50,50,50,47,50,4,50,49,46,2
Personal & Household Goods,39,39,39,37,39,39,39,37,39,2,39,38,37,1
Chemicals,38,38,38,38,38,38,38,38,38,1,38,38,37,3
Financial Services,0,36,36,32,36,36,36,31,36,6,36,36,31,5


In [42]:
#### WHICH SECTORS are recession-proof ?
### Consumer goods, Retail
### Healthcare
### Utilies (water and electricity etc) 
### IT, TECH: CMG, ELC, FPT
df.loc[df['sector']=='Distribution Services']


,ticker,sector,industry,website
4,ABS,Distribution Services,Wholesale Distributors,bitagco.com
22,ASP,Distribution Services,Wholesale Distributors,anphapetrol.com
40,BTT,Distribution Services,Wholesale Distributors,benthanhtsc.com.vn
60,COM,Distribution Services,Wholesale Distributors,comeco.vn
61,CRC,Distribution Services,Wholesale Distributors,createcapital.vn
66,CTF,Distribution Services,Wholesale Distributors,cityford.com.vn
73,DAG,Distribution Services,Wholesale Distributors,dag.com.vn
83,DGW,Distribution Services,Electronics Distributors,thegioiso.vn
110,EVG,Distribution Services,Wholesale Distributors,everland.vn
118,FRT,Distribution Services,Electronics Distributors,frt.vn
